In [ ]:
!pip install pandas numpy scipy scikit-learn

In [2]:
import pandas as pd
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
#Configurar diretório e carregar dados

!wget http://files.grouplens.org/datasets/movielens/ml-latest-small.zip
!unzip ml-latest-small.zip

dataset_dir = "ml-latest-small"

#Carregar avaliações
ratings = pd.read_csv(f"{dataset_dir}/ratings.csv")  #userId, movieId, rating, timestamp

#Carregar filmes
movies = pd.read_csv(f"{dataset_dir}/movies.csv")    #movieId, title, genres

print("Dados carregados!")
print(f"Total de avaliações: {len(ratings)}")
print(f"Total de filmes: {len(movies)}")

In [ ]:
#Mapear IDs para índices e criar matriz esparsa

#Mapear userId e movieId para índices contínuos
user_mapper = {uid: i for i, uid in enumerate(ratings['userId'].unique())}
movie_mapper = {mid: i for i, mid in enumerate(ratings['movieId'].unique())}

#Mapear IDs originais para índices
ratings['user_idx'] = ratings['userId'].map(user_mapper)
ratings['movie_idx'] = ratings['movieId'].map(movie_mapper)

#Criar matriz esparsa (usuários x filmes)
ratings_sparse = csr_matrix(
    (ratings['rating'], (ratings['user_idx'], ratings['movie_idx']))
)

#Mapeamento inverso (para retornar IDs originais)
movie_inv_mapper = {i: mid for mid, i in movie_mapper.items()}
user_inv_mapper = {i: uid for uid, i in user_mapper.items()}

print("Matriz esparsa criada!")
print(f"Shape da matriz: {ratings_sparse.shape}")

In [ ]:
#Calcular similaridade entre filmes

#Transpor para ter filmes nas linhas (necessário para filme-filme similarity)
#dense_output=False mantém saída esparsa
movie_similarity = cosine_similarity(ratings_sparse.T, dense_output=False)

print("Matriz de similaridade entre filmes calculada!")

In [6]:
#Função para recomendar filmes similares
#Retorna os top_n filmes mais similares a um movie_id dado

def recommend_movies(movie_id, top_n=5):
    #Obter índice do filme
    if movie_id not in movie_mapper:
        print(f"Filme {movie_id} não encontrado.")
        return []

    movie_idx = movie_mapper[movie_id]

    #Similaridade do filme com todos os outros
    sim_scores = movie_similarity[movie_idx].toarray().flatten()

    #Criar série com movie_idx como índice
    sim_series = pd.Series(sim_scores, index=range(len(sim_scores)))

    #Top N similares (excluindo o próprio)
    top_indices = sim_series.sort_values(ascending=False).iloc[1:top_n+1].index

    #Recuperar títulos e scores
    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        score = sim_series[idx]
        recommendations.append((title, score))

    return recommendations

In [ ]:
#Testar recomendações para um filme

#Exemplo: movieId = 1 (Toy Story 1995)
movie_id = 1
top_movies = recommend_movies(movie_id, top_n=5)

print(f"Top 5 filmes similares a '{movies.loc[movies.movieId==movie_id,'title'].values[0]}':")
for title, score in top_movies:
    print(f"{title} — similaridade {score:.3f}")

In [8]:
#Função para recomendar para um usuário
#Retorna recomendações para um usuário baseado em ratings passados e similaridade filme-filme

def recommend_for_user(user_id, top_n=5):
    if user_id not in user_mapper:
        print(f"Usuário {user_id} não encontrado.")
        return []

    user_idx = user_mapper[user_id]

    #Filmes avaliados pelo usuário
    user_ratings = ratings_sparse[user_idx, :].toarray().flatten()
    rated_indices = np.where(user_ratings > 0)[0]

    #Score agregado inicial
    scores = np.zeros(ratings_sparse.shape[1])

    #Para cada filme avaliado, acumular score ponderado pela similaridade
    for idx in rated_indices:
        sim_scores = movie_similarity[idx].toarray().flatten()
        scores += sim_scores * user_ratings[idx]

    #Remover filmes já avaliados
    scores[rated_indices] = 0

    #Top N recomendações
    top_indices = np.argsort(scores)[::-1][:top_n]

    recommendations = []
    for idx in top_indices:
        mid = movie_inv_mapper[idx]
        title = movies.loc[movies.movieId == mid, 'title'].values[0]
        recommendations.append((title, scores[idx]))

    return recommendations

In [ ]:
#Testar recomendações para um usuário

user_id = 26
top_user_movies = recommend_for_user(user_id, top_n=5)

print(f"Top 5 recomendações para o usuário {user_id}:")
for title, score in top_user_movies:
    print(f"{title} — score {score:.3f}")